In [1]:
import sys, re
from transformers import pipeline
from pypdf import PdfReader

c:\Users\ander\Documents\Projetos\Aulas\proadi-sus-ciencia-de-dados-ia\aulas\Aulas\DL - Agentes\DL - Rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Extrai texto do PDF
reader = PdfReader("data/anvisa/rdc/2018/rdc_243_julho_2018.pdf")
paginas = [p.extract_text() for p in reader.pages if p.extract_text()]

In [18]:
if not paginas:
    print("PDF sem texto pra extrair. Se for um scan (imagem), faça OCR antes.")
    raise SystemExit(1)

In [24]:
# Pipeline de NER multilíngue
ner = pipeline(
    "token-classification",
    model="pierreguillou/ner-bert-base-cased-pt-lenerbr",
    aggregation_strategy="simple",
    device=-1  # força CPU
)


Device set to use cpu


In [25]:
# NER por página (a pipeline aceita lista e processa em batch)
resultados = ner(paginas)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [27]:
# Impressão por página
for i, ents in enumerate(resultados, start=1):
    print(f"\n# PÁGINA {i}")
    if ents:
        for e in ents:
            print(f"{e['entity_group']}\t{e['word']}\t{e['score']:.2f}")
    else:
        print("— sem entidades detectadas —")


# PÁGINA 1
ORGANIZACAO	Ministério da Saúde	0.91
ORGANIZACAO	Agência Nacional de Vigilância Sanitária	1.00
LEGISLACAO	R	0.94
LEGISLACAO	##ES	0.93
LEGISLACAO	##OL	0.91
LEGISLACAO	##U	0.74
LEGISLACAO	##Ç	0.80
LEGISLACAO	##ÃO DA DIRETORIA COLEGIADA	0.85
LEGISLACAO	R	0.94
LEGISLACAO	##DC Nº 243	0.98
TEMPO	26 DE JULHO DE 2018	1.00
ORGANIZACAO	Agência Nacional de Vigilância Sanitária	1.00
LEGISLACAO	art. 15, III e IV aliado ao art. 7°, III e IV, da Lei n° 9. 782	0.98
TEMPO	26 de janeiro de 1999	1.00
LEGISLACAO	art. 53, V, § § 1° e 3° do Regimento Interno	1.00
LEGISLACAO	An	1.00
LEGISLACAO	##ex	0.99
LEGISLACAO	##o I da Resolução da Diretoria Colegiada	0.99
LEGISLACAO	R	0.95
LEGISLACAO	##DC n° 61	0.97
TEMPO	3 de fevereiro de 2016	1.00
TEMPO	17 de julho de 2018	1.00

# PÁGINA 2
LEGISLACAO	An	1.00
LEGISLACAO	##ex	1.00
LEGISLACAO	##os I e II da Instrução Normativa n° 28	1.00
TEMPO	26 de julho de 2018	1.00

# PÁGINA 3
LEGISLACAO	An	1.00
LEGISLACAO	##ex	1.00
LEGISLACAO	##os I e II da Instrução Norm

In [28]:
texto_total = "\n".join(paginas)
padrao_ato = re.compile(
    r"\b(?:RDC|IN|Instrução\s+Normativa)\s*(?:n[ºo\.]*\s*)?(\d{1,4})\s*/\s*(\d{2,4})",
    flags=re.IGNORECASE
)
atos = [m.group(0).strip() for m in padrao_ato.finditer(texto_total)]

print("\n# ATOS NORMATIVOS (regex)")
if atos:
    for a in atos:
        print(a)
else:
    print("— nenhum padrão de ato normativo encontrado —")


# ATOS NORMATIVOS (regex)
— nenhum padrão de ato normativo encontrado —
